```mermaid
graph LR
A[Select Appropriate Model] --> B[Data Preparation Training data, validation data]
B --> C[Parameter Selection EPOC, Batch Size]
C --> D[Model Training]
D --> E[Model Evaluation]
E --> F[Save the created fine tuned Model Locally]
F --> G[Use the fine Tune Model Locally]
```

In [ ]:
%pip install datasets transformers rouge-score nltk
%pip install sentencepiece
%pip install transformers
%pip install torch
%pip install rich[jupyter]

In [1]:
# Importing libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os


In [2]:
# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [3]:
path  = "review_data_Product.csv" #change appropriately
df = pd.read_csv(path)
df.head()

,Review,Product
0,Solid entry level Kindle. Great for kids. Gift...,Kindle
1,This make an excellent ebook reader. Don't exp...,Ebook Reader
2,I use this every day on my commute. Great batt...,Unknown
3,Everything is great except that i can't read e...,Kindle Books
4,I never used kindle before. My wife gifted me ...,Kindle


In [4]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)


cpu


In [5]:
# rich: for a better display on terminal
from rich.table import Column, Table
from rich import box
from rich.console import Console
# define a rich console logger
console = Console(record=True)
# to display dataframe in ASCII format
def display_df(df):
    """display dataframe in ASCII format"""
    console = Console()
    table = Table(
        Column("source_text", justify="center"),
        Column("target_text", justify="center"),
        title="Sample Data",
        pad_edge=False,
        box=box.ASCII,
    )
    for i, row in enumerate(df.values.tolist()):
        table.add_row(row[0], row[1])
    console.print(table)
# training logger to log training progress
training_logger = Table(
    Column("Epoch", justify="center"),
    Column("Steps", justify="center"),
    Column("Loss", justify="center"),
    title="Training Status",
    pad_edge=False,
    box=box.ASCII,
)


In [6]:
class ProductExtractor(Dataset):
    """
    Creating a custom dataset for reading the dataset and
    loading it into the dataloader to pass it to the
    neural network for finetuning the model
    """
    def __init__(
        self, dataframe, tokenizer, source_len, target_len, source_text, target_text
    ):
        """
        Initializes a Dataset class
        Args:
            dataframe (pandas.DataFrame): Input dataframe
            tokenizer (transformers.tokenizer): Transformers tokenizer
            source_len (int): Max length of source text
            target_len (int): Max length of target text
            source_text (str): column name of source text
            target_text (str): column name of target text
        """
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = target_len
        self.target_text = self.data[target_text]
        self.source_text = self.data[source_text]
    def __len__(self):
        """returns the length of dataframe"""
        return len(self.target_text)
    def __getitem__(self, index):
        """return the input ids, attention masks and target ids"""
        source_text = str(self.source_text[index])
        target_text = str(self.target_text[index])
        # cleaning data so as to ensure data is in string type
        source_text = " ".join(source_text.split())
        target_text = " ".join(target_text.split())
        source = self.tokenizer.batch_encode_plus(
            [source_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        target = self.tokenizer.batch_encode_plus(
            [target_text],
            max_length=self.summ_len,
            pad_to_max_length=True,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        source_ids = source["input_ids"].squeeze()
        source_mask = source["attention_mask"].squeeze()
        target_ids = target["input_ids"].squeeze()
        target_mask = target["attention_mask"].squeeze()
        return {
            "source_ids": source_ids.to(dtype=torch.long),
            "source_mask": source_mask.to(dtype=torch.long),
            "target_ids": target_ids.to(dtype=torch.long),
            "target_ids_y": target_ids.to(dtype=torch.long),
        }


In [7]:
# let's define model parameters specific to T5
training_params = {
    "MODEL": "t5-base",  # model_type: t5-base/t5-large
    "TRAIN_BATCH_SIZE": 8,  # training batch size
    "VALID_BATCH_SIZE": 8,  # validation batch size
    "TRAIN_EPOCHS": 3,  # number of training epochs
    "VAL_EPOCHS": 1,  # number of validation epochs
    "LEARNING_RATE": 1e-4,  # learning rate
    "MAX_SOURCE_TEXT_LENGTH": 512,  # max length of source text
    "MAX_TARGET_TEXT_LENGTH": 50,  # max length of target text
    "SEED": 42,  # set seed for reproducibility
}


In [8]:
def T5Trainer(
    dataframe, source_text, target_text, model_params, output_dir="./review_outputs/"
):
    """
    T5 trainer
    """
    # Set random seeds and deterministic pytorch for reproducibility
    torch.manual_seed(model_params["SEED"])  # pytorch random seed
    np.random.seed(model_params["SEED"])  # numpy random seed
    torch.backends.cudnn.deterministic = True
    # logging
    console.log(f"""[Model]: Loading {model_params["MODEL"]}...\n""")
    # tokenzier for encoding the text
    tokenizer = T5Tokenizer.from_pretrained(model_params["MODEL"])
    # Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary.
    # Further this model is sent to device (GPU/TPU) for using the hardware.
    model = T5ForConditionalGeneration.from_pretrained(model_params["MODEL"])
    model = model.to(device)
    # logging
    console.log(f"[Data]: Reading data...\n")
    # Importing the raw dataset
    dataframe = dataframe[[source_text, target_text]]
    display_df(dataframe.head(2))
    # Creation of Dataset and Dataloader
    # Defining the train size. So 80% of the data will be used for training and the rest for validation.
    train_size = 0.8
    train_dataset = dataframe.sample(frac=train_size, random_state=model_params["SEED"])
    val_dataset = dataframe.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)
    console.print(f"FULL Dataset: {dataframe.shape}")
    console.print(f"TRAIN Dataset: {train_dataset.shape}")
    console.print(f"TEST Dataset: {val_dataset.shape}\n")
    # Creating the Training and Validation dataset for further creation of Dataloader
    training_set = ProductExtractor(
        train_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    val_set = ProductExtractor(
        val_dataset,
        tokenizer,
        model_params["MAX_SOURCE_TEXT_LENGTH"],
        model_params["MAX_TARGET_TEXT_LENGTH"],
        source_text,
        target_text,
    )
    # Defining the parameters for creation of dataloaders
    train_params = {
        "batch_size": model_params["TRAIN_BATCH_SIZE"],
        "shuffle": True,
        "num_workers": 0,
    }
    val_params = {
        "batch_size": model_params["VALID_BATCH_SIZE"],
        "shuffle": False,
        "num_workers": 0,
    }
    # Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)
    # Defining the optimizer that will be used to tune the weights of the network in the training session.
    optimizer = torch.optim.Adam(
        params=model.parameters(), lr=model_params["LEARNING_RATE"]
    )
    # Training loop
    console.log(f"[Initiating Fine Tuning]...\n")
    for epoch in range(model_params["TRAIN_EPOCHS"]):
        train(epoch, tokenizer, model, device, training_loader, optimizer)
    console.log(f"[Saving Model]...\n")
    # Saving the model after training
    path = os.path.join(output_dir, "model_files")
    model.save_pretrained(path)
    tokenizer.save_pretrained(path)
    # evaluating test dataset
    console.log(f"[Initiating Validation]...\n")
    for epoch in range(model_params["VAL_EPOCHS"]):
        predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
        final_df = pd.DataFrame({"Generated Text": predictions, "Actual Text": actuals})
        final_df.to_csv(os.path.join(output_dir, "predictions_prod_senti.csv"))
    console.save_text(os.path.join(output_dir, "review_logs.txt"))
    console.log(f"[Validation Completed.]\n")
    console.print(
        f"""[Model] Model saved @ {os.path.join(output_dir, "review_model_files")}\n"""
    )
    console.print(
        f"""[Validation] Generation on Validation data saved @ {os.path.join(output_dir,'review_predictions.csv')}\n"""
    )
    console.print(f"""[Logs] Logs saved @ {os.path.join(output_dir,'review_logs.txt')}\n""")


In [9]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    """
    Function to be called for training with the parameters passed from main function
    """
    model.train()
    for _, data in enumerate(loader, 0):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["source_ids"].to(device, dtype=torch.long)
        mask = data["source_mask"].to(device, dtype=torch.long)
        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]
        if _ % 10 == 0:
            training_logger.add_row(str(epoch), str(_), str(loss))
            console.print(training_logger)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [10]:
def validate(epoch, tokenizer, model, device, loader):
  """
  Function to evaluate model for predictions
  """
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in enumerate(loader, 0):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['source_ids'].to(device, dtype = torch.long)
          mask = data['source_mask'].to(device, dtype = torch.long)
          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          if _%10==0:
              console.print(f'Completed {_}')
          predictions.extend(preds)
          actuals.extend(target)
  return predictions, actuals


In [12]:
# T5 accepts prefix of the task to be performed:
# Since we are summarizing, let's add summarize to source text as a prefix
df["Review"] = "Extract Product: " + df["Review"]
T5Trainer(
    dataframe=df,		#[:500], INPUT FIRST 500 ROWS.
    source_text="Review",
    target_text="Product",
    model_params=training_params,
    output_dir="review_outputs",
)


[12:19:49] [Model]: Loading t5-base...                                                              ]8;id=529048;file:///var/folders/df/4n57lpzd4z986szq__z25w840000gn/T/ipykernel_42437/289667992.py\289667992.py]8;;\:]8;id=428990;file:///var/folders/df/4n57lpzd4z986szq__z25w840000gn/T/ipykernel_42437/289667992.py#12\12]8;;\
                                                                                                                   

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

                      Training Status                       
+----------------------------------------------------------+
|Epoch | Steps |                    Loss                   |
|------+-------+-------------------------------------------|
|  0   |   0   | tensor(9.3073, grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(4.4282, grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(0.6241, grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(1.1970, grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(1.1373, grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.2461, grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(0.6894, grad_fn=<NllLossBackward0>)|
|  1   |  10   | tensor(1.0690, grad_fn=<NllLossBackward0>)|
|  1   |  20   | tensor(0.0335, grad_fn=<NllLossBackward0>)|
|  1   |  30   | tensor(0.6509, grad_fn=<NllLossBackward0>)|
|  1   |  40   | tensor(0.1858, grad_fn=<NllLossBackward0>)|
|  1   |  50   | tensor(1.3012, grad_fn=<NllLossBackward0>)|
|  2   |   0   | tensor(1.3968, grad_fn=<NllLossBackward0>)|
+----------------------------------------------------------+

                      Training Status                       
+----------------------------------------------------------+
|Epoch | Steps |                    Loss                   |
|------+-------+-------------------------------------------|
|  0   |   0   | tensor(9.3073, grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(4.4282, grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(0.6241, grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(1.1970, grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(1.1373, grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.2461, grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(0.6894, grad_fn=<NllLossBackward0>)|
|  1   |  10   | tensor(1.0690, grad_fn=<NllLossBackward0>)|
|  1   |  20   | tensor(0.0335, grad_fn=<NllLossBackward0>)|
|  1   |  30   | tensor(0.6509, grad_fn=<NllLossBackward0>)|
|  1   |  40   | tensor(0.1858, grad_fn=<NllLossBackward0>)|
|  1   |  50   | tensor(1.3012, grad_fn=<NllLossBackward0>)|
|  2   |   0   | tensor(1.3968, grad_fn=<NllLossBackward0>)|
|  2   |  10   | tensor(0.4913, grad_fn=<NllLossBackward0>)|
+----------------------------------------------------------+

                      Training Status                       
+----------------------------------------------------------+
|Epoch | Steps |                    Loss                   |
|------+-------+-------------------------------------------|
|  0   |   0   | tensor(9.3073, grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(4.4282, grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(0.6241, grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(1.1970, grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(1.1373, grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.2461, grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(0.6894, grad_fn=<NllLossBackward0>)|
|  1   |  10   | tensor(1.0690, grad_fn=<NllLossBackward0>)|
|  1   |  20   | tensor(0.0335, grad_fn=<NllLossBackward0>)|
|  1   |  30   | tensor(0.6509, grad_fn=<NllLossBackward0>)|
|  1   |  40   | tensor(0.1858, grad_fn=<NllLossBackward0>)|
|  1   |  50   | tensor(1.3012, grad_fn=<NllLossBackward0>)|
|  2   |   0   | tensor(1.3968, grad_fn=<NllLossBackward0>)|
|  2   |  10   | tensor(0.4913, grad_fn=<NllLossBackward0>)|
|  2   |  20   | tensor(0.2258, grad_fn=<NllLossBackward0>)|
+----------------------------------------------------------+

                      Training Status                       
+----------------------------------------------------------+
|Epoch | Steps |                    Loss                   |
|------+-------+-------------------------------------------|
|  0   |   0   | tensor(9.3073, grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(4.4282, grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(0.6241, grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(1.1970, grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(1.1373, grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.2461, grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(0.6894, grad_fn=<NllLossBackward0>)|
|  1   |  10   | tensor(1.0690, grad_fn=<NllLossBackward0>)|
|  1   |  20   | tensor(0.0335, grad_fn=<NllLossBackward0>)|
|  1   |  30   | tensor(0.6509, grad_fn=<NllLossBackward0>)|
|  1   |  40   | tensor(0.1858, grad_fn=<NllLossBackward0>)|
|  1   |  50   | tensor(1.3012, grad_fn=<NllLossBackward0>)|
|  2   |   0   | tensor(1.3968, grad_fn=<NllLossBackward0>)|
|  2   |  10   | tensor(0.4913, grad_fn=<NllLossBackward0>)|
|  2   |  20   | tensor(0.2258, grad_fn=<NllLossBackward0>)|
|  2   |  30   | tensor(0.0303, grad_fn=<NllLossBackward0>)|
+----------------------------------------------------------+

                      Training Status                       
+----------------------------------------------------------+
|Epoch | Steps |                    Loss                   |
|------+-------+-------------------------------------------|
|  0   |   0   | tensor(9.3073, grad_fn=<NllLossBackward0>)|
|  0   |  10   | tensor(4.4282, grad_fn=<NllLossBackward0>)|
|  0   |  20   | tensor(0.6241, grad_fn=<NllLossBackward0>)|
|  0   |  30   | tensor(1.1970, grad_fn=<NllLossBackward0>)|
|  0   |  40   | tensor(1.1373, grad_fn=<NllLossBackward0>)|
|  0   |  50   | tensor(1.2461, grad_fn=<NllLossBackward0>)|
|  1   |   0   | tensor(0.6894, grad_fn=<NllLossBackward0>)|
|  1   |  10   | tensor(1.0690, grad_fn=<NllLossBackward0>)|
|  1   |  20   | tensor(0.0335, grad_fn=<NllLossBackward0>)|
|  1   |  30   | tensor(0.6509, grad_fn=<NllLossBackward0>)|
|  1   |  40   | tensor(0.1858, grad_fn=<NllLossBackward0>)|
|  1   |  50   | tensor(1.3012, grad_fn=<NllLossBackward0>)|
|  2   |   0   | tensor(1.3968, grad_fn=<NllLossBackward0>)|
|  2   |  10   | tensor(0.4913, grad_fn=<NllLossBackward0>)|
|  2   |  20   | tensor(0.2258, grad_fn=<NllLossBackward0>)|
|  2   |  30   | tensor(0.0303, grad_fn=<NllLossBackward0>)|
|  2   |  40   | tensor(0.4487, grad_fn=<NllLossBackward0>)|
+----------------------------------------------------------+

In [ ]:
console.log(f"Saving the model ... \n")
path=os.path.join(".", "model_files")


model.save_pretrained(path)
tokenizer.save_pretrained(path)

tokenizer = T5Tokenizer.from_pretrained("./review_outputs/model_files",local_files_only=True)
model = T5ForConditionalGeneration.from_pretrained("./review_outputs/model_files",local_files_only=True)

In [ ]:
ids = tokenizer("Extract Product :This laptop has good battery life. \
It works quite fast i must say", return_tensors="pt").input_ids
generated_ids = model.generate(
              input_ids = ids,
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
print(preds)
       
